<a href="https://colab.research.google.com/github/saadiqbal96/cybersecuritytrivia/blob/main/ProjectNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================
# ProjectNotebook.ipynb Template
# =============================

# ---------------------------------
# 1️⃣ DATA LOADING & EXPLORATION
# ---------------------------------

# Imports
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Transformations (including augmentation)
train_transforms = transforms.Compose([
    transforms.RandomRotation(15),  # Example augmentation
    transforms.ToTensor(),
    # You can add normalization if needed
])

test_transforms = transforms.Compose([
    transforms.ToTensor()
])

# Load datasets
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transforms)

# Create DataLoaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

# Explore dataset
print(f"Training data size: {len(train_set)}")
print(f"Example image shape: {train_set[0][0].shape}")

# Show example image
image, label = test_set[0]
plt.imshow(image.permute(1, 2, 0))  # Convert from CxHxW to HxWxC for plotting
plt.title(f"Label: {label}")
plt.show()


# ---------------------------------
# 2️⃣ MODEL DESIGN & TRAINING
# ---------------------------------
import torch.nn as nn
import torch.optim as optim

# Define model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=1)

model = CNNModel()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
train_losses = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

# Plot training loss
plt.plot(range(1, num_epochs+1), train_losses, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.title('Training Loss per Epoch')
plt.show()


# ---------------------------------
# 3️⃣ MODEL TESTING & EVALUATION
# ---------------------------------
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total * 100
print(f"Test Accuracy: {accuracy:.2f}%")

# Analysis (replace with your 3+ sentences)
print("""
Analysis:
1. The model achieved an accuracy of {:.2f}% on the test set.
2. This accuracy meets/falls short of the hypothetical baseline model (e.g., 45%).
3. Recommendation: Based on results, building a custom object detection solution might be feasible, but consider pre-trained models for better performance.
""".format(accuracy))

# Save model
torch.save(model.state_dict(), 'cnn_model.pth')
print("Model saved as cnn_model.pth")